In [40]:
%reload_ext autoreload
%autoreload 2
from itertools import chain
from os import listdir
from collections import defaultdict
from NaiveBayes import MultinomialNaiveBayes
from preprocessing import bag_of_words

In [41]:
folds = sorted([(int(a[4:]), []) for a in listdir('pu1/') if a[0] != '.'])

In [42]:
for fold in folds:
    d = 'pu1/part{}/'.format(fold[0])
    for f_name in listdir(d):
        with open(d + f_name) as f:
            lines = [line.rstrip('\n') for line in f]
            subj = lines[0].lstrip('Subject: ')
            text = lines[2:][0]
            fold[1].append([f_name.find('spm') >= 0, subj, text])
folds = [l for (ignore, l) in folds]

In [43]:
bigfold = list(chain(*folds[:-1]))

In [44]:
texts = [bag_of_words(a[2]) for a in bigfold]
subjects = [bag_of_words(a[1]) for a in bigfold]
y = [a[0] for a in bigfold]
x = []
for (i, t) in enumerate(texts):
    x.append({k: t.get(k, 0) + subjects[i].get(k, 0) * 1.2 for k in set(t) | set(subjects[i])})
    
t_text = [bag_of_words(a[2]) for a in folds[-1]]
t_subj = [bag_of_words(a[1]) for a in folds[-1]]
t_y = [a[0] for a in folds[-1]]
t_x = []
for (i, t) in enumerate(t_subj):
    t_x.append({ k: t_text[i].get(k, 0) + t.get(k, 0) * 1.2 for k in set(t) | set(t_text[i]) })

In [45]:
bayes = MultinomialNaiveBayes(0.5, 5.0)
bayes.fit(x, y)

In [46]:
def score(X, y):
        res = [0, 0]
        for (i,x) in enumerate(X):
            res[y[i]] += bayes.predict(X[i]) != y[i]
        return res

In [47]:
score(x, y)

[16, 4]

In [39]:
score(t_x, t_y)

False True
False True
False True
False True
True False
True False
True False
False True
False True
False False
False True
True False
True False
True False
False True
True False
True False
True False
True False
False True
True False
True False
True False
False True
True False
